# 🖥️ CROCO tutorial

This tutorial will show how to run a 3D simulation with output from the CROCO model.

## Example setup

We start with loading the relevant modules and the data.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

import parcels

data_folder = parcels.download_example_dataset("CROCOidealized_data")
ds_fields = xr.open_dataset(data_folder / "CROCO_idealized.nc")

ds_fields.load();  # Preload data to speed up access

The simulation will be a very simple, idealised flow: a purely zonal flow over a sloping bottom. This flow (which is somewhat unrealistic of course) nicely showcases that particles stay on their initial depth levels, even though the sigma-layers slope down. 

This flow has been created by first running the example from the [Shelf front example on the CROCO website](https://croco-ocean.gitlabpages.inria.fr/croco_doc/model/model.test_cases.shelfront.html). Then, we took the restart file are manually replaced all the `u`-velocities with `1` m/s and all the `v`-velocities with `0` m/s. This way we get a purely zonal flow. We then started a new simulation from the restart file, and CROCO then automatically calculated the `w` velocities to match the new zonal field. We saved the `time=0` snapshot from this new run and use it below.

Now we create a FieldSet object using the `FieldSet.from_croco()` method. Note that CROCO is a C-grid (with similar indexing at MITgcm).

```{note}
In the code below, we use the `w` velocity field for vertical velocity. However, it is unclear whether this is always the right choice. CROCO (and ROMS) also output an `omega` field, which may be more appropriate to use. The idealised simulation below only works when using `w`, though. In other simulations, it is recommended to test whether `omega` provides more realistic results. See https://github.com/OceanParcels/Parcels/discussions/1728 for more information.
```

In [ ]:
fields = {
    "U": ds_fields["u"],
    "V": ds_fields["v"],
    "W": ds_fields["w"],
    "h": ds_fields["h"],
    "zeta": ds_fields["zeta"],
    "Cs_w": ds_fields["Cs_w"],
}

coords = ds_fields[["x_rho", "y_rho", "s_w", "time"]]
ds_fset = parcels.convert.croco_to_sgrid(fields=fields, coords=coords)

fieldset = parcels.FieldSet.from_sgrid_conventions(ds_fset)

# Add the critical depth (`hc`) as a constant to the fieldset
fieldset.add_constant("hc", ds_fields.hc)

Now we can use this Fieldset to advect particles as we would normally do. Note that the particle depths should be provided in (negative) meters, not in sigma-coordinates.

In [ ]:
X, Z = np.meshgrid(
    [40e3, 80e3, 120e3],
    [100, -10, -130, -250, -400, -850, -1400, -1550],
)
Y = np.ones(X.size) * 98000


def DeleteParticle(particles, fieldset):
    any_error = particles.state >= 50
    particles[any_error].state = parcels.StatusCode.Delete


pset = parcels.ParticleSet(
    fieldset=fieldset, pclass=parcels.Particle, lon=X, lat=Y, z=Z
)

outputfile = parcels.ParticleFile(
    store="croco_particles3D.zarr",
    outputdt=np.timedelta64(5000, "s"),
)

pset.execute(
    [parcels.kernels.AdvectionRK4_3D_CROCO, DeleteParticle],
    runtime=np.timedelta64(50_000, "s"),
    dt=np.timedelta64(100, "s"),
    output_file=outputfile,
)

Now we plot the particle trajectories below. Note that the particles stay on their initial depth levels, even though the sigma-layers slope down. Also note that particles released above the surface (where depth >0) or below the bathymetry are not advected (due to the `DeleteParticle` kernel).

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
ds = xr.open_zarr("croco_particles3D.zarr")

ax.plot(X / 1e3, Z, "k.", label="Initial positions")
ax.plot(ds.lon.T / 1e3, ds.z.T, ".-")

for z in ds_fields.s_w.values:
    ax.plot(
        fieldset.h.grid.lon[0, :] / 1e3,
        fieldset.h.data[0, 0, 25, :] * z,
        "k",
        linewidth=0.5,
    )
ax.set_xlabel("X [km]")
ax.set_xlim(30, 170)
ax.set_ylabel("Depth [m]")
ax.set_title("Particles in idealized CROCO velocity field using AdvectionRK4_3D_CROCO")
plt.tight_layout()
plt.show()

### A CROCO simulation with no vertical velocities

It may be insightful to compare this 3D run with the `AdvectionRK4_3D_CROCO` kernel with a run where the vertical velocity (`W`) is set to zero. In that case, the particles will not stay on their initial depth levels but instead follow sigma-layers.

In [ ]:
fieldset_noW = parcels.FieldSet.from_sgrid_conventions(ds_fset)
fieldset_noW.W.data[:] = 0.0
fieldset_noW.add_constant("hc", ds_fields.hc)

X, Z = np.meshgrid(
    [40e3, 80e3, 120e3],
    [100, -10, -130, -250, -400, -850, -1400, -1550],
)
Y = np.ones(X.size) * 100

pset_noW = parcels.ParticleSet(
    fieldset=fieldset_noW, pclass=parcels.Particle, lon=X, lat=Y, z=Z
)

outputfile = parcels.ParticleFile(
    store="croco_particles_noW.zarr", outputdt=np.timedelta64(5000, "s")
)

pset_noW.execute(
    [parcels.kernels.AdvectionRK4_3D_CROCO, DeleteParticle],
    runtime=np.timedelta64(50_000, "s"),
    dt=np.timedelta64(100, "s"),
    output_file=outputfile,
)

fig, ax = plt.subplots(1, 1, figsize=(6, 4))
ds = xr.open_zarr("croco_particles_noW.zarr")

ax.plot(X / 1e3, Z, "k.", label="Initial positions")
ax.plot(ds.lon.T / 1e3, ds.z.T, ".-")

for z in ds_fields.s_w.values:
    ax.plot(
        fieldset.h.grid.lon[0, :] / 1e3,
        fieldset.h.data[0, 0, 25, :] * z,
        "k",
        linewidth=0.5,
    )
ax.set_xlabel("X [km]")
ax.set_xlim(30, 170)
ax.set_ylabel("Depth [m]")
ax.set_title("Particles in idealized CROCO velocity field with W=0")
plt.tight_layout()
plt.show()

## The algorithms used

When using Croco data, Parcels needs to convert the particles.depth to sigma-coordinates, before doing any interpolation. This is done with the `convert_z_to_sigma_croco()`  function. Interpolating onto a Field is then done like:

```python
def SampleTempCroco(particles, fieldset):
    """Sample temperature field on a CROCO sigma grid by first converting z to sigma levels."""
    sigma = convert_z_to_sigma_croco(fieldset, particles.time, particles.z, particles.lat, particles.lon, particles)
    particles.temp = fieldset.T[particles.time, sigma, particles.lat, particles.lon, particles]
```

For Advection, you will need to use the `AdvectionRK4_3D_CROCO` kernel, which works slightly different from the normal 3D advection kernel because it converts the vertical velocity in sigma-units. The conversion from depth to sigma is done at every time step, using the `convert_z_to_sigma_croco()` function.

In particular, the following algorithm is used (note that the RK4 version is slightly more complex than this Euler-Forward version, but the idea is identical). Also note that the vertical velocity is linearly interpolated here, which gives much better results than the default C-grid interpolation.

```python
sigma = particles.z / fieldset.h[particles.time, 0, particles.lat, particles.lon]

sigma_levels = convert_z_to_sigma_croco(fieldset, particles.time, particles.z, particles.lat, particles.lon, particles)
(u, v) = fieldset.UV[time, sigma_levels, particle.lat, particle.lon, particle]
w = fieldset.W[time, sigma_levels, particle.lat, particle.lon, particle]

# scaling the w with the sigma level of the particle
w_sigma = w * sigma / fieldset.h[particles.time, 0, particles.lat, particles.lon]

lon_new = particles.lon + u*particles.dt
lat_new = particles.lat + v*particles.dt

# calculating new sigma level
sigma_new = sigma + w_sigma*particles.dt 

# converting back from sigma to depth, at _new_ location
depth_new = sigma_new * fieldset.h[particles.time, 0, lat_new, lon_new]
```